<a href="https://colab.research.google.com/github/Kaua-Rbs/Deep-Learning-From-A-To-Z-With-Pytorch-And-Python/blob/main/Project_9_Regression_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 9: Used Cars Regression - Cross-Validation

## Importing Libraries

In [1]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 6.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch.nn as nn
from skorch import NeuralNetRegressor
import torch
from sklearn.model_selection import cross_val_score
import time
torch.__version__

'2.3.0+cu121'

## Database

In [3]:
torch.manual_seed(123)

In [4]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

In [5]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [6]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [7]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [8]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values.reshape(-1, 1)

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])], remainder = 'passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

In [10]:
previsores = previsores.astype('float32')
preco_real = preco_real.astype('float32')

## Model Structuring

In [11]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(316, 158)
        self.dense1 = nn.Linear(158, 158)
        self.dense2 = nn.Linear(158, 1)
        self.activation = nn.ReLU()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dense2(X)
        return X

In [12]:
regressor_sklearn = NeuralNetRegressor(module = regressor_torch,
                                       criterion = torch.nn.L1Loss,
                                       optimizer = torch.optim.Adam,
                                       max_epochs = 100,
                                       batch_size = 300,
                                       train_split = False)

## Cross-Validation

In [13]:
resultados = cross_val_score(regressor_sklearn, previsores, preco_real, cv = 5,
                             scoring = 'neg_mean_absolute_error')

  epoch    train_loss      dur
-------  ------------  -------
      1     3597.2495  11.0762
      2     2986.3451  6.1169
      3     2867.4092  7.2936
      4     2812.8217  6.1197
      5     2762.9784  7.4378
      6     2718.9486  6.1613
      7     2675.2359  7.2643
      8     2650.8238  6.0318
      9     2634.7752  7.4337
     10     2590.4902  6.1654
     11     2555.8405  7.3906
     12     2582.0268  6.1575
     13     2539.5562  7.4293
     14     2527.1217  6.0151
     15     2515.0421  7.5403
     16     2499.9160  6.1861
     17     2477.1310  7.4250
     18     2496.7486  6.1721
     19     2481.5153  7.1897
     20     2464.2015  6.1061
     21     2478.5805  7.4057
     22     2475.8196  6.1416
     23     2451.9517  7.4208
     24     2443.9680  6.1072
     25     2446.0519  7.2985
     26     2443.9942  6.0845
     27     2438.4151  7.5274
     28     2427.6177  6.0873
     29     2427.8761  7.3010
     30     2429.6653  6.1727
     31     2423.6220  7.5874
     32

In [1]:
media = -resultados.mean()
desvio = resultados.std()

NameError: name 'resultados' is not defined

In [15]:
media, desvio

(-2310.124365234375, 36.37499103270745)